# Generic linear regression model

In [3]:
import numpy as np
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf #for manipulating DataFrames with SQL style queries
import ast #for literally evaluating strings taht were jsons

In [10]:
df = pd.read_csv('media_objects.csv')
df.drop([0], axis = 0, inplace = True) #Drop column descriptors
#Drop unnecessary columns
df.drop(['media_object_id','timestamp', 'media_url'], axis = 1, inplace = True)

In [8]:
#RESET INDEX LATER

In [11]:
df.head()

caption  \
1  @CheeseGrotto on a crisp fall eve, in a Manhat...   
2  Our #cheesesafe is a beautiful, simple gift fo...   
3  This rich bloomy rind cheese was made in a @Ch...   
4  Our #cheesesafe with an awesome leather handle...   
5  The goats want to know.....where's the @Cheese...   

                                            comments  comments_count  \
1  {'data': [{'timestamp': '2014-11-11T00:20:51+0...             1.0   
2                                                NaN             0.0   
3  {'data': [{'timestamp': '2016-09-09T03:22:34+0...             2.0   
4                                                NaN             0.0   
5                                                NaN             0.0   

  media_type  like_count engagement impressions reach saved  video_views  
1      IMAGE         8.0        NaN         NaN   NaN   NaN          NaN  
2      IMAGE         6.0        NaN         NaN   NaN   NaN          NaN  
3      IMAGE        10.0        NaN         NaN   NaN   NaN          NaN  
4      IMAGE         8.0        NaN         NaN   NaN   NaN          NaN  
5      IMAGE         1.0        NaN         NaN   NaN   NaN          NaN

### Engineering features

In [6]:
#Get count of hashtags
def get_ht(line):
    return sum([1 for word in line.split() if word[0] == '#'])

#Get hashtags from the caption column
fr_cap = [get_ht(line) for line in df['caption'].astype(str).values]

#Get hashtags from the first comment in the comments column
#Comments are in reverse chronological order

#Does not include hashtags by commenter:
#fr_com = [get_ht(ast.literal_eval(line)['data'][-1]['text'])\
#                              if type(line) == str else 0 for line in df['comments'].values ]

#This includes hashtags by followers. Presumption is that most are by poster
fr_com = [sum([get_ht(comment['text']) for comment in ast.literal_eval(line)['data']])\
                              if type(line) == str else 0 for line in df['comments'].values ]

df['hashtag_count'] = np.array(fr_cap) + np.array(fr_com)

In [ ]:
#Get yes/no for starting emoji

In [84]:
test_text = df['caption'].values[-3]

In [86]:
test_text[0]

'🍓'

In [80]:
df['caption'].values

array(['@CheeseGrotto on a crisp fall eve, in a Manhattan garden. #cheese #cheesegrotto #elegance  http://kck.st/1qnfB7U',
       'Our #cheesesafe is a beautiful, simple gift for that rustic #cheese lover we all know. @cheesegrotto @kickstarter http://kck.st/1qnfB7U',
       "This rich bloomy rind cheese was made in a @CheeseGrotto. It's raw cow's milk. I call it, The Lowing. #cheesegrotto #cheese #rawmilkcheese",
       ...,
       '🍓 It. me. Cheese, chocolate-covered strawberries, and sparkling wine to ring in the New Year.\u2063\n🍾 How are you celebrating?',
       'Fresh year. Fresh perspective. Fresh cheese. 👋 Try your hand at fresh cheesemaking in 2020, link in bio.',
       '🌶 This is grilled cheese weather.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n☝️Browse our Cooking with Cheese section of the blog for melty grilled cheese wonders like this @gruyere_aop cheese, roasted pepper, kale, and @brinsjam chili pepper jam sandwich. You know where the link is...'],
      dtype=object)

In [ ]:


#Extend each list in the 'from captions' list by the values in the 'from comments' list
[fr_cap[i].extend(fr_com_i) for i, fr_com_i in enumerate(fr_com)]

#Add this as a column to the df
df['hashtags'] = fr_cap

In [ ]:
## FROM STORY STORY


#Index the df with datetimes
df.set_index(pd.to_datetime(df['timestamp']), inplace = True)
#Reset time zone to US Eastern time
df.index = df.index.tz_convert('US/Eastern')

#Change remaining columns to int type
df = df.astype(int)

In [ ]:
## FROM HASHTAGS HASHTAGS

def get_ht(line):
    return [word[1:] for word in line.split() if word[0] == '#']

#Get hashtags from the caption column
fr_cap = [get_ht(line) for line in df['caption'].astype(str).values]
#Get hashtags from the first comment in the comments column
fr_com = [get_ht(ast.literal_eval(line)['data'][0]['text'])\
                              if type(line) == str else [] for line in df['comments'].values ]
#Extend each list in the 'from captions' list by the values in the 'from comments' list
[fr_cap[i].extend(fr_com_i) for i, fr_com_i in enumerate(fr_com)]

#Add this as a column to the df
df['hashtags'] = fr_cap